# generation_kwargs["max_new_tokens"] = max_new_tokensRLHF

## Reward Modelling

In [ ]:
#!pip install peft trl weaviate-client rouge_score nltk

In [1]:
import time
import os
import torch
from peft import LoraConfig, TaskType, get_peft_model, PeftConfig
from trl import RewardConfig, RewardTrainer
from typing import List
import numpy as np
import pandas as pd
import evaluate

from transformers import (
    AutoModelForSeq2SeqLM,
    AutoModelForCausalLM,
    AutoModelForSequenceClassification,
    AutoTokenizer,
    pipeline,
)
from datasets import load_from_disk

from trl import PPOTrainer, PPOConfig, AutoModelForSeq2SeqLMWithValueHead, AutoModelForCausalLMWithValueHead
from trl import create_reference_model
from trl.core import LengthSampler
import uuid
from tqdm import tqdm


TESTING = False

PRECISION_NAME = 'float16'
DEVICE = "cuda"
CHOSEN_MODEL = "microsoft/phi-1_5"
# "microsoft/phi-1_5" #"bigscience/mt0-small" # "google/flan-t5-large" "stabilityai/stablelm-2-zephyr-1_6b"
RANDOM_SEED = 42
RUN_ID = "db816086ae284b9ea8cb56fc799897a8" # SFT MODEL # uuid.uuid4().hex  

LORA_PARAM_TARGET_MODULES = {
    "bigscience/mt0-small": ["q", "v"],
    "microsoft/phi-1_5": ["q_proj", "v_proj"],
    "microsoft/Phi-3-mini-4k-instruct": ["qkv_proj"],
    "microsoft/Phi-3-medium-4k-instruct": ["qkv_proj"],
}

RM_LORA_PARAM_R = 32
RM_LORA_PARAM_ALPHA = 64
RM_LORA_PARAM_TARGET_MODULES = LORA_PARAM_TARGET_MODULES[CHOSEN_MODEL] + [
    "dense",
    "out_proj",
]
RM_TRAIN_BATCH_SIZE = 5
RM_LEARNING_RATE = 1e-5
RM_TRAIN_DATA_RUN_ID = "b592e2304b5c4b41a33236871ed4e195"

RL_LORA_PARAM_R = 32
RL_LORA_PARAM_ALPHA = 64
RL_LORA_PARAM_TARGET_MODULES = LORA_PARAM_TARGET_MODULES[CHOSEN_MODEL]
RL_TRAIN_BATCH_SIZE = 2
RL_TRAIN_MINI_BATCH_SIZE = 1
RL_LEARNING_RATE = 1.41e-7
RL_N_EPOCHS = 1

/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.25.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
import sys
from pathlib import Path

path = Path.cwd().parent.absolute()
nyx_path = f'{path}/'
print(nyx_path)
sys.path.append(nyx_path)

/home/ubuntu/demerzel/


In [52]:
from nyx.evaluation import quantitative_comparison
from nyx.data_generation.evaluators import AILabelEvaluator

from nyx.utils import (
    precision_enumerator,
    print_number_of_trainable_model_parameters,
    get_task_type,
    round_dictionary_values,
)
from nyx.constants import (
    COMPARISON_DATA_PATH,
    SFT_DATA_OUTPUT_PATH,
    COMMON_OUTPUT_PATHS,
    SFT_PEFT_MERGED_MODEL_PATH,
    SFT_PEFT_ADAPTER_PATH,
    RM_TRAIN_DATA_PATH,
    RM_OUTPUT_DIR,
    RM_PEFT_ADAPTER_PATH,
    RM_PEFT_MERGED_MODEL_PATH,
    METRICS_PATH,
)


common_output_path = COMMON_OUTPUT_PATHS.format(RUN_ID=RUN_ID)

SFT_PEFT_ADAPTER_PATH = SFT_PEFT_ADAPTER_PATH.format(
    COMMON_OUTPUT_PATHS=common_output_path
)
SFT_PEFT_MERGED_MODEL_PATH = SFT_PEFT_MERGED_MODEL_PATH.format(
    COMMON_OUTPUT_PATHS=common_output_path
)

RM_OUTPUT_DIR = RM_OUTPUT_DIR.format(COMMON_OUTPUT_PATHS=common_output_path)
RM_PEFT_ADAPTER_PATH = RM_PEFT_ADAPTER_PATH.format(
    COMMON_OUTPUT_PATHS=common_output_path
)
RM_PEFT_MERGED_MODEL_PATH = RM_PEFT_MERGED_MODEL_PATH.format(
    COMMON_OUTPUT_PATHS=common_output_path
)

# if generated data from a different run needs to be utilised.
if RM_TRAIN_DATA_RUN_ID is not None:
    rm_common_path = COMMON_OUTPUT_PATHS.format(RUN_ID=RM_TRAIN_DATA_RUN_ID)
    RM_TRAIN_DATA_PATH = RM_TRAIN_DATA_PATH.format(COMMON_OUTPUT_PATHS=rm_common_path)
else:  # utilise current run_id
    RM_TRAIN_DATA_PATH = RM_TRAIN_DATA_PATH.format(
        COMMON_OUTPUT_PATHS=common_output_path
    )

METRICS_PATH = METRICS_PATH.format(COMMON_OUTPUT_PATHS=common_output_path)

PRECISION = precision_enumerator(PRECISION_NAME)
PRECISION

torch.float16

### Load model and data

In [4]:
# Original dataset with train, test and validation
comparison_dataset = load_from_disk(COMPARISON_DATA_PATH)

# Train dataset with generated AI labels
comparison_train_dataset = load_from_disk(RM_TRAIN_DATA_PATH)

if TESTING is True:
    comparison_dataset = comparison_dataset.filter(
        lambda example, index: index % 10 == 0, with_indices=True
    )
comparison_dataset

DatasetDict({
    train: Dataset({
        features: ['subreddit', 'post', 'choice', 'candidate_summary_1', 'candidate_summary_2'],
        num_rows: 92858
    })
    validation: Dataset({
        features: ['subreddit', 'post', 'choice', 'candidate_summary_1', 'candidate_summary_2'],
        num_rows: 83802
    })
})

In [5]:
try:
    sft_model = AutoModelForSeq2SeqLM.from_pretrained(
        SFT_PEFT_MERGED_MODEL_PATH,
        torch_dtype=PRECISION,
        device_map="auto",
        # device=DEVICE,
        # attn_implementation="flash_attention_2",
    )
except ValueError:
    sft_model = AutoModelForCausalLM.from_pretrained(
        SFT_PEFT_MERGED_MODEL_PATH,
        torch_dtype=PRECISION,
        device_map="auto",
        # device=DEVICE,
#         attn_implementation="flash_attention_2",
    )

tokenizer = AutoTokenizer.from_pretrained(CHOSEN_MODEL, padding_side='left')
tokenizer.pad_token = (
    tokenizer.pad_token if tokenizer.pad_token is not None else tokenizer.eos_token  # '<|endoftext|>'
)
tokenizer.pad_token

'<|endoftext|>'

In [6]:
sft_model.config.pad_token_id = sft_model.config.eos_token_id
# tokenizer.pad_token_id = tokenizer.eos_token_id

In [7]:
def create_summary_cols(example):
    start_prompt = "Summarize the following reddit post.\n\n"
    end_prompt = "\n\nSummary: "
    example['summary_prompts_1'] = (
        start_prompt + example["post"] + end_prompt + example["candidate_summary_1"]
    )
    example['summary_prompts_2'] = (
        start_prompt + example["post"] + end_prompt + example["candidate_summary_2"]
    )
    return example


comparison_train_dataset = comparison_train_dataset.map(create_summary_cols)

# tokenized_train_dataset['train']['summary_prompts_1'][0]


def prepare_for_reward_modelling(example, hf_baseline: bool = False):
    choice_column = example["choice"] if hf_baseline is True else example["ai_choice"]
    # ai_choice is based on index choice 0 ==summary 1, choice 1 == summary 2
    example["accepted_summary"] = (
        example['summary_prompts_2']
        if choice_column == example["constant_col"]
        else example['summary_prompts_1']
    )
    example["rejected_summary"] = (
        example['summary_prompts_1']
        if choice_column == example["constant_col"]
        else example['summary_prompts_2']
    )
    return example


comparison_train_dataset = comparison_train_dataset.map(prepare_for_reward_modelling)

In [8]:
comparison_train_dataset

DatasetDict({
    train: Dataset({
        features: ['subreddit', 'post', 'choice', 'candidate_summary_1', 'candidate_summary_2', 'ai_choice', 'ai_choice_for_prompt', 'constant_col', 'nth_retry', 'ordered_prompt_used_to_predict', 'reversed_prompt_used_to_predict', 'incorrect_prediction', 'summary_prompts_1', 'summary_prompts_2', 'accepted_summary', 'rejected_summary'],
        num_rows: 10000
    })
})

### Data preparation
Encoder-Decoder specific line 29<br>
Sampling from a range of start and end prompts could help robustness in the RM model.

In [9]:
def tokenize_function(example):
    # start_prompt = "Summarize the following reddit post.\n\n"
    # end_prompt = "\n\nSummary: "
    # prompt = [start_prompt + dialogue + end_prompt for dialogue in example["post"]]
    accepted = tokenizer(
        example["accepted_summary"],
        padding=True,
        # padding='max_length',
        truncation=True,
        return_tensors="pt",
    ).to(torch.device(DEVICE))
    example["input_ids_chosen"] = accepted.input_ids
    example["attention_mask_chosen"] = accepted.attention_mask
    rejected = tokenizer(
        example["rejected_summary"],
        padding=True,
        # padding='max_length',
        truncation=True,
        return_tensors="pt",
    ).to(torch.device(DEVICE))
    example["input_ids_rejected"] = rejected.input_ids
    example["attention_mask_rejected"] = rejected.attention_mask

    example["labels"] = tokenizer(
        [str(choice) for choice in example["ai_choice"]],
        padding=True,
        # padding='max_length',
        truncation=True,
        return_tensors="pt",
    ).input_ids.to(torch.device(DEVICE))

    if 'causal' in sft_model.config.architectures[0].lower() and sft_model.config.architectures[0] != 'PhiForCausalLM':
        example["decoder_input_ids"] = sft_model._shift_right(example["labels"])
    return example


reward_modelling_train_dataset = comparison_train_dataset.map(
    tokenize_function, batched=True
)
reward_modelling_train_dataset = reward_modelling_train_dataset.remove_columns(
    [
        'subreddit',
        'post',
        'choice',
        'candidate_summary_1',
        'candidate_summary_2',
        # 'prompts',
        'ai_choice',
        'constant_col',
        'accepted_summary',
        'rejected_summary',
        'summary_prompts_2',
        'summary_prompts_1',
    ]  # 'is_match',
)
reward_modelling_train_dataset

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['ai_choice_for_prompt', 'nth_retry', 'ordered_prompt_used_to_predict', 'reversed_prompt_used_to_predict', 'incorrect_prediction', 'input_ids_chosen', 'attention_mask_chosen', 'input_ids_rejected', 'attention_mask_rejected', 'labels'],
        num_rows: 10000
    })
})

The below model is the pretrained SFT model with an additional head for classification.

In [10]:
base_reward_model = AutoModelForSequenceClassification.from_pretrained(
    SFT_PEFT_MERGED_MODEL_PATH, torch_dtype=PRECISION
)

lora_config = LoraConfig(
    # Determines the size of LoRA matrices. x*r * r*y = x*y
    r=RM_LORA_PARAM_R,
    # scaling coefficient. Paper mentions it is important because the adjustments are small compared
    # to the rest of the model.
    lora_alpha=RM_LORA_PARAM_ALPHA,
    # Variable target_modules determines what layers are fine-tuned, see architecture above.
    # Simplest case scenario based on the original paper.
    # The parameters / layers of the new head need to be enabled for training.
    target_modules=RM_LORA_PARAM_TARGET_MODULES,
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.SEQ_CLS,
)

rm_peft_model = get_peft_model(base_reward_model, lora_config)
print(print_number_of_trainable_model_parameters(rm_peft_model))
# base_reward_model

Some weights of PhiForSequenceClassification were not initialized from the model checkpoint at microsoft/phi-1_5 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


trainable model parameters: 9441280
all model parameters: 1322807296
percentage of trainable model parameters: 0.71%


In [11]:
reward_modelling_train_dataset['train']
# print(tokenizer.eos_token_id)
base_reward_model.config.pad_token_id = tokenizer.eos_token_id
base_reward_model.config.pad_token = tokenizer.pad_token
# base_reward_model.config

In [12]:
training_args = RewardConfig(
    output_dir=RM_OUTPUT_DIR,
    # auto_find_batch_size=True,
    per_device_train_batch_size=RM_TRAIN_BATCH_SIZE,
    save_steps=500,
    learning_rate=RM_LEARNING_RATE,  # Higher learning rate than full fine-tuning.
    logging_steps=1,
    max_steps=len(reward_modelling_train_dataset['train']) // RM_TRAIN_BATCH_SIZE,
)

reward_trainer = RewardTrainer(  # trainer class child
    model=rm_peft_model,
    args=training_args,  # trainerarguments child
    tokenizer=tokenizer,
    train_dataset=reward_modelling_train_dataset['train'],
    # num_labels=1, # Regression
)

# for _, param in peft_model.named_parameters():
#         # all_model_params += param.numel()
#         param.requires_grad = True
# print(print_number_of_trainable_model_parameters(peft_model))

/home/ubuntu/.local/lib/python3.10/site-packages/trl/trainer/reward_trainer.py:175: UserWarning: When using RewardDataCollatorWithPadding, you should set `max_length` in RewardConfig. It will be set to `512` by default, but you should do it yourself in the future.
  warnings.warn(
/home/ubuntu/.local/lib/python3.10/site-packages/trl/trainer/reward_trainer.py:192: UserWarning: When using RewardDataCollatorWithPadding, you should set `remove_unused_columns=False` in your RewardConfig we have set it for you, but you should do it yourself in the future.
  warnings.warn(
max_steps is given, it will override any value given in num_train_epochs


In [13]:
start = time.time()

reward_trainer.train()
end = time.time()

duration = end - start
print(end)
print(f"Training for 1 epoch took {round(duration, 2)} seconds to execute.")

reward_trainer.model.save_pretrained(RM_PEFT_ADAPTER_PATH)
tokenizer.save_pretrained(RM_PEFT_ADAPTER_PATH)

You're using a CodeGenTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/home/ubuntu/.local/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:2906: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(
Could not estimate the number of tokens of the input, floating-point operations will not be computed


Step,Training Loss
1,0.681200
2,0.000000
3,0.000000
4,0.000000
5,0.000000
6,0.000000
7,0.000000
8,0.000000
9,0.000000
10,0.000000


/home/ubuntu/.local/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:2906: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(
/home/ubuntu/.local/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:2906: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(
/home/ubuntu/.local/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:2906: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(


1722948543.0084193
Training for 1 epoch took 5600.07 seconds to execute.


('./experiments/db816086ae284b9ea8cb56fc799897a8/models/reward-modelling/peft-checkpoint-local/tokenizer_config.json',
 './experiments/db816086ae284b9ea8cb56fc799897a8/models/reward-modelling/peft-checkpoint-local/special_tokens_map.json',
 './experiments/db816086ae284b9ea8cb56fc799897a8/models/reward-modelling/peft-checkpoint-local/vocab.json',
 './experiments/db816086ae284b9ea8cb56fc799897a8/models/reward-modelling/peft-checkpoint-local/merges.txt',
 './experiments/db816086ae284b9ea8cb56fc799897a8/models/reward-modelling/peft-checkpoint-local/added_tokens.json',
 './experiments/db816086ae284b9ea8cb56fc799897a8/models/reward-modelling/peft-checkpoint-local/tokenizer.json')

### Reload and evaluate RM model

### Merge and save RM model (with base model)
So that, the sentiment pipe warning is eliminated.

In [27]:
# base_reward_model.active_adapter

<bound method PeftAdapterMixin.active_adapter of PhiForSequenceClassification(
  (model): PhiModel(
    (embed_tokens): Embedding(51200, 2048)
    (embed_dropout): Dropout(p=0.0, inplace=False)
    (layers): ModuleList(
      (0-23): 24 x PhiDecoderLayer(
        (self_attn): PhiAttention(
          (q_proj): lora.Linear(
            (base_layer): Linear(in_features=2048, out_features=2048, bias=True)
            (lora_dropout): ModuleDict(
              (default): Dropout(p=0.05, inplace=False)
            )
            (lora_A): ModuleDict(
              (default): Linear(in_features=2048, out_features=16, bias=False)
            )
            (lora_B): ModuleDict(
              (default): Linear(in_features=16, out_features=2048, bias=False)
            )
            (lora_embedding_A): ParameterDict()
            (lora_embedding_B): ParameterDict()
            (lora_magnitude_vector): ModuleDict()
          )
          (k_proj): Linear(in_features=2048, out_features=2048, bias=True

In [13]:
# Merging and saving the model which is trained all the way (i.e., utilising all of the data).
from peft import AutoPeftModelForSequenceClassification

merged_rm_model = AutoPeftModelForSequenceClassification.from_pretrained(RM_PEFT_ADAPTER_PATH, device_map='auto', torch_dtype=PRECISION)

merged_rm_model = merged_rm_model.merge_and_unload()
merged_rm_model.save_pretrained(RM_PEFT_MERGED_MODEL_PATH)

merged_rm_model#.to(torch.device(DEVICE))
RM_PEFT_MERGED_MODEL_PATH

Some weights of PhiForSequenceClassification were not initialized from the model checkpoint at microsoft/phi-1_5 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


'./experiments/db816086ae284b9ea8cb56fc799897a8/models/reward-modelling/merged-with-peft-adapter'

In [ ]:
### Loop this over [base, 500, 1000, 5000, 10_000, 50_000, 100_000] to calculate the pairwise accuracy


# peft_rm_model = PeftModel.from_pretrained(base_reward_model,
#                                        f'{rm_peft_model_path}',
#                                        torch_dtype=PRECISION,
#                                        is_trainable=False)

# peft_rm_model.to(torch.device('mps'))
# print('PEFT trained RM is loaded.')

### _Comparing HF and AIF data (RM model generated)_
Utilising the train test split function to select a random 15% of the validation set to compare HF and AIF labels.

In [14]:
rm_eval_data = comparison_dataset['validation'].train_test_split(
    test_size=0.02, seed=RANDOM_SEED # roughly 1'600 examples with 2%
)
rm_eval_data['test']
print(len(rm_eval_data['test']))

merged_rm_model.config.pad_token_id = tokenizer.eos_token_id
merged_rm_model.config.pad_token = tokenizer.pad_token

def get_rm_probabilities(
    col: str, rm_model_to_evaluate=merged_rm_model, batch_size: int = 10
) -> List[List[str]]:
    candidate_probabilities = []
    for i in range(0, len(rm_eval_data['test']), batch_size):
        candidate = rm_model_to_evaluate(
            tokenizer(
                rm_eval_data['test'][col][i: i + batch_size],
                # padding="max_length",
                padding=True,
                truncation=True,
                return_tensors="pt",
            ).input_ids.to(DEVICE)
        )
    
        candidate_probabilities.extend(candidate.logits.softmax(dim=-1).tolist())
    return candidate_probabilities


candidate_1_probabilities = get_rm_probabilities('candidate_summary_1')
candidate_2_probabilities = get_rm_probabilities('candidate_summary_2')
print(len(candidate_1_probabilities), len(candidate_2_probabilities))

We detected that you are passing `past_key_values` as a tuple and this is deprecated and will be removed in v4.43. Please use an appropriate `Cache` class (https://huggingface.co/docs/transformers/v4.41.3/en/internal/generation_utils#transformers.Cache)


1677
1677 1677


In [15]:
# Taking the first item in probabilities will yield the probability of being a chosen summary.
rm_eval_data['test'] = rm_eval_data['test'].add_column(
    name="candidate_1_preference_probability",
    column=[item[0] for item in candidate_1_probabilities],
)
rm_eval_data['test'] = rm_eval_data['test'].add_column(
    name="candidate_2_preference_probability",
    column=[item[0] for item in candidate_2_probabilities],
)


def get_rm_labels(example):
    example["rm_choice"] = (
        0
        if example["candidate_1_preference_probability"]
        >= example["candidate_2_preference_probability"]
        else 1
    )
    example["is_match"] = 1 if example["choice"] == example["rm_choice"] else 0
    return example


# Apply the function to each example in the dataset
rm_eval_data['test'] = rm_eval_data['test'].map(get_rm_labels)

# Calculate the mean value of the 'is_match' feature
rm_aggreement_mean_value = np.round(np.mean(rm_eval_data['test']["is_match"]) * 100, 2)
print(
    f"The Reward Model (RM) is in agreement with the annotator provided labels: {rm_aggreement_mean_value}% of the times."
)
rm_eval_data

The Reward Model (RM) is in agreement with the annotator provided labels: 51.64% of the times.


DatasetDict({
    train: Dataset({
        features: ['subreddit', 'post', 'choice', 'candidate_summary_1', 'candidate_summary_2'],
        num_rows: 82125
    })
    test: Dataset({
        features: ['subreddit', 'post', 'choice', 'candidate_summary_1', 'candidate_summary_2', 'candidate_1_preference_probability', 'candidate_2_preference_probability', 'rm_choice', 'is_match'],
        num_rows: 1677
    })
})

In [16]:
evaluator = AILabelEvaluator(data_to_evaluate=rm_eval_data, run_id=RM_TRAIN_DATA_RUN_ID if RM_TRAIN_DATA_RUN_ID is not None else RUN_ID)

evaluator.compute_metrics(data_split='test', predicted_col='rm_choice')

The prompt driven, instruct model generated feedback labels are in agreement with the annotator provided labels: 51.64% of the times.
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       811
           1       0.52      1.00      0.68       866

    accuracy                           0.52      1677
   macro avg       0.26      0.50      0.34      1677
weighted avg       0.27      0.52      0.35      1677

tp: 866, fp: 811
fn: 0, tn: 0
MCC is 0.0.


/usr/lib/python3/dist-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/lib/python3/dist-packages/sklearn/metrics/_classification.py:846: RuntimeWarning: invalid value encountered in scalar divide
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)


## Reinforcement Learning

In [14]:
dataset = load_from_disk(SFT_DATA_OUTPUT_PATH)
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'subreddit', 'title', 'post', 'summary'],
        num_rows: 116722
    })
    test: Dataset({
        features: ['id', 'subreddit', 'title', 'post', 'summary'],
        num_rows: 6553
    })
    validation: Dataset({
        features: ['id', 'subreddit', 'title', 'post', 'summary'],
        num_rows: 6447
    })
})

In [18]:
if TESTING is True:
    dataset = dataset.filter(
        lambda example, index: index % 4680 == 0, with_indices=True
    )
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'subreddit', 'title', 'post', 'summary'],
        num_rows: 116722
    })
    test: Dataset({
        features: ['id', 'subreddit', 'title', 'post', 'summary'],
        num_rows: 6553
    })
    validation: Dataset({
        features: ['id', 'subreddit', 'title', 'post', 'summary'],
        num_rows: 6447
    })
})

In [18]:
rl_lora_config = LoraConfig(
    r=RL_LORA_PARAM_R,  # Rank
    lora_alpha=RL_LORA_PARAM_ALPHA,
    target_modules=RL_LORA_PARAM_TARGET_MODULES,
    lora_dropout=0.05,
    bias="none",
    task_type=get_task_type(model=sft_model),
)


rl_peft_model = get_peft_model(sft_model, rl_lora_config)
print(print_number_of_trainable_model_parameters(rl_peft_model))

trainable model parameters: 6291456
all model parameters: 1424562176
percentage of trainable model parameters: 0.44%


In [19]:
# A transformer model with an additional scalar output for each token which can be used as a value function in reinforcement learning
loading_class = AutoModelForCausalLMWithValueHead if 'causal' in sft_model.config.architectures[0].lower() else AutoModelForSeq2SeqLMWithValueHead
ppo_model = loading_class.from_pretrained(
    rl_peft_model, torch_dtype=PRECISION, is_trainable=True, device_map='auto'
)

print(f'PPO model has {print_number_of_trainable_model_parameters(ppo_model)}\n')
print(ppo_model)
ppo_model#.to(torch.device(DEVICE))

PPO model has trainable model parameters: 6293505
all model parameters: 1424564225
percentage of trainable model parameters: 0.44%

AutoModelForCausalLMWithValueHead(
  (pretrained_model): PeftModelForCausalLM(
    (base_model): LoraModel(
      (model): PhiForCausalLM(
        (model): PhiModel(
          (embed_tokens): Embedding(51200, 2048)
          (embed_dropout): Dropout(p=0.0, inplace=False)
          (layers): ModuleList(
            (0-23): 24 x PhiDecoderLayer(
              (self_attn): PhiAttention(
                (q_proj): lora.Linear(
                  (base_layer): Linear(in_features=2048, out_features=2048, bias=True)
                  (lora_dropout): ModuleDict(
                    (default): Dropout(p=0.05, inplace=False)
                  )
                  (lora_A): ModuleDict(
                    (default): Linear(in_features=2048, out_features=32, bias=False)
                  )
                  (lora_B): ModuleDict(
                    (default): Linear(in_f

AutoModelForCausalLMWithValueHead(
  (pretrained_model): PeftModelForCausalLM(
    (base_model): LoraModel(
      (model): PhiForCausalLM(
        (model): PhiModel(
          (embed_tokens): Embedding(51200, 2048)
          (embed_dropout): Dropout(p=0.0, inplace=False)
          (layers): ModuleList(
            (0-23): 24 x PhiDecoderLayer(
              (self_attn): PhiAttention(
                (q_proj): lora.Linear(
                  (base_layer): Linear(in_features=2048, out_features=2048, bias=True)
                  (lora_dropout): ModuleDict(
                    (default): Dropout(p=0.05, inplace=False)
                  )
                  (lora_A): ModuleDict(
                    (default): Linear(in_features=2048, out_features=32, bias=False)
                  )
                  (lora_B): ModuleDict(
                    (default): Linear(in_features=32, out_features=2048, bias=False)
                  )
                  (lora_embedding_A): ParameterDict()
               

The below function could also be adapted to sample from a variety of prompts to improve exploration and improve
robustness.

In [20]:
def tokenize_for_rl(sample):
    # Wrap each dialogue with the instruction.
    prompt = f"""
Summarize the following reddit post.

{sample["post"]}

Summary:
"""
    sample["input_ids"] = tokenizer.encode(prompt)

    # Requirement for PPO library.
    sample["query"] = tokenizer.decode(sample["input_ids"])
    return sample


# Tokenize each dialogue.
dataset = dataset.map(tokenize_for_rl, batched=False)
dataset.set_format(type="torch")
dataset

Map:   0%|          | 0/116722 [00:00<?, ? examples/s]

Map:   0%|          | 0/6553 [00:00<?, ? examples/s]

Map:   0%|          | 0/6447 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'subreddit', 'title', 'post', 'summary', 'input_ids', 'query'],
        num_rows: 116722
    })
    test: Dataset({
        features: ['id', 'subreddit', 'title', 'post', 'summary', 'input_ids', 'query'],
        num_rows: 6553
    })
    validation: Dataset({
        features: ['id', 'subreddit', 'title', 'post', 'summary', 'input_ids', 'query'],
        num_rows: 6447
    })
})

In [36]:
# dataset['train']['query'][:5]

["\nSummarize the following reddit post.\n\nNot sure if this belongs here but it's worth a try. \n\nBackstory:\nWhen I (f/22) went through my first real breakup 2 years ago because he needed space after a year of dating roand  it effected me more than I thought. It was a horrible time in my life due to living with my mother and finally having the chance to cut her out of my life. I can admit because of it was an emotional wreck and this guy was stable and didn't know how to deal with me. We ended by him avoiding for a month or so after going to a festival with my friends. When I think back I wish he just ended. So after he ended it added my depression I suffered but my friends helped me through it and I got rid of everything from him along with cutting contact. \n\nNow: Its been almost 3 years now and I've gotten better after counselling and mild anti depressants. My mother has been out of my life since then so there's been alot of progress. Being stronger after learning some lessons t

In [21]:
ref_model = create_reference_model(ppo_model)
ref_model#.to(torch.device(DEVICE))
print(
    f'Reference model parameters to be updated:\n{print_number_of_trainable_model_parameters(ref_model)}\n'
)

Reference model parameters to be updated:
trainable model parameters: 0
all model parameters: 1424564225
percentage of trainable model parameters: 0.00%



In [22]:
def collator(data):
    return dict((key, [d[key] for d in data]) for key in data[0])


test_data = [
    {"key1": "value1", "key2": "value2", "key3": "value3"},
    {"key1": "value2", "key2": "value3", "key3": "value4"},
]
print(f'Collator input: {test_data}')
print(f'Collator output: {collator(test_data)}')

Collator input: [{'key1': 'value1', 'key2': 'value2', 'key3': 'value3'}, {'key1': 'value2', 'key2': 'value3', 'key3': 'value4'}]
Collator output: {'key1': ['value1', 'value2'], 'key2': ['value2', 'value3'], 'key3': ['value3', 'value4']}


In [23]:
config = PPOConfig(
    # Name of model to use - used only for tracking purposes
    model_name=CHOSEN_MODEL,
    learning_rate=RL_LEARNING_RATE,
    ppo_epochs=RL_N_EPOCHS,
    mini_batch_size=RL_TRAIN_MINI_BATCH_SIZE,
    batch_size=RL_TRAIN_BATCH_SIZE,
)

ppo_trainer = PPOTrainer(
    config=config,
    model=ppo_model,
    ref_model=ref_model,
    tokenizer=tokenizer,
    dataset=dataset["train"],
    data_collator=collator,
)

In [24]:
sentiment_pipe = pipeline(
    "sentiment-analysis", model=merged_rm_model, tokenizer=tokenizer, #device=DEVICE
)

In [36]:
merged_rm_model.config

PhiConfig {
  "_name_or_path": "microsoft/phi-1_5",
  "architectures": [
    "PhiForSequenceClassification"
  ],
  "attention_dropout": 0.0,
  "bos_token_id": null,
  "embd_pdrop": 0.0,
  "eos_token_id": null,
  "hidden_act": "gelu_new",
  "hidden_size": 2048,
  "initializer_range": 0.02,
  "intermediate_size": 8192,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 2048,
  "model_type": "phi",
  "num_attention_heads": 32,
  "num_hidden_layers": 24,
  "num_key_value_heads": 32,
  "pad_token": "<|endoftext|>",
  "pad_token_id": 50256,
  "partial_rotary_factor": 0.5,
  "qk_layernorm": false,
  "resid_pdrop": 0.0,
  "rope_scaling": null,
  "rope_theta": 10000.0,
  "tie_word_embeddings": false,
  "torch_dtype": "float16",
  "transformers_version": "4.43.4",
  "use_cache": true,
  "vocab_size": 50295
}

In [35]:
ppo_model.config.pad_token_id = tokenizer.pad_token_id
ppo_model.config.pad_token = tokenizer.pad_token
merged_rm_model.config.pad_token_id = tokenizer.pad_token_id
merged_rm_model.config.pad_token = tokenizer.pad_token

In [39]:
output_min_length = 100
output_max_length = 600
output_length_sampler = LengthSampler(output_min_length, output_max_length)

preferred_summary_index = 0

generation_kwargs = {"min_length": 5, "temperature": 0.6, "do_sample": False} #"top_k": 0.0, "top_p": 1.0 # "do_sample": True

reward_kwargs = {
    "top_k": None,  # Return all scores.
    "function_to_apply": "none",  # Raw logits without softmax.
    "batch_size": RL_TRAIN_BATCH_SIZE,
}

max_ppo_steps = 5


for step, batch in tqdm(enumerate(ppo_trainer.dataloader)):
    # print(step, batch)
    # Break when you reach max_steps.
    if step >= max_ppo_steps:
        break

    prompt_tensors = batch["input_ids"]

    # Get response from FLAN-T5/PEFT LLM.
    summary_tensors = []

    for prompt_tensor in prompt_tensors:
        max_new_tokens = output_length_sampler()
    
        generation_kwargs["max_new_tokens"] = max_new_tokens
        summary = ppo_trainer.generate(prompt_tensor, **generation_kwargs)

        summary_tensors.append(summary.squeeze()[-max_new_tokens:])

    # This needs to be called "response".
    batch["response"] = [tokenizer.decode(r.squeeze()) for r in summary_tensors]

    # Compute reward outputs.
    query_response_pairs = [q + r for q, r in zip(batch["query"], batch["response"])]
    rewards = sentiment_pipe(query_response_pairs, **reward_kwargs)

    # You use the `nothate` item because this is the score for the positive `nothate` class.
    reward_tensors = [
        torch.tensor(reward[preferred_summary_index]["score"]) for reward in rewards
    ]

    # Run PPO step.
    stats = ppo_trainer.step(prompt_tensors, summary_tensors, reward_tensors)
    ppo_trainer.log_stats(stats, batch, reward_tensors)

    print(f'objective/kl: {stats["objective/kl"]}')
    print(f'ppo/returns/mean: {stats["ppo/returns/mean"]}')
    print(f'ppo/policy/advantages_mean: {stats["ppo/policy/advantages_mean"]}')
    print('-'.join('' for x in range(100)))

1it [00:20, 20.95s/it]

objective/kl: nan
ppo/returns/mean: nan
ppo/policy/advantages_mean: nan
---------------------------------------------------------------------------------------------------


2it [00:48, 24.80s/it]

objective/kl: nan
ppo/returns/mean: nan
ppo/policy/advantages_mean: nan
---------------------------------------------------------------------------------------------------


3it [01:13, 24.95s/it]

objective/kl: nan
ppo/returns/mean: nan
ppo/policy/advantages_mean: nan
---------------------------------------------------------------------------------------------------


4it [01:25, 19.75s/it]

objective/kl: nan
ppo/returns/mean: nan
ppo/policy/advantages_mean: nan
---------------------------------------------------------------------------------------------------


5it [01:47, 21.53s/it]

objective/kl: nan
ppo/returns/mean: nan
ppo/policy/advantages_mean: nan
---------------------------------------------------------------------------------------------------


### _Evaluate RL model_

In [55]:
# Merging and saving the model which is trained all the way (i.e., utilising all of the data).

ppo_trainer.model.save_pretrained(RM_PEFT_ADAPTER_PATH)
tokenizer.save_pretrained(RM_PEFT_ADAPTER_PATH)

merged_rm_model = AutoPeftModelForSequenceClassification.from_pretrained(RM_PEFT_ADAPTER_PATH, device_map='auto', torch_dtype=PRECISION)


merged_rm_model = merged_rm_model.merge_and_unload()
merged_rm_model.save_pretrained(RM_PEFT_MERGED_MODEL_PATH)

merged_rm_model#.to(torch.device(DEVICE))
RM_PEFT_MERGED_MODEL_PATH

PhiForCausalLM(
  (model): PhiModel(
    (embed_tokens): Embedding(51200, 2048)
    (embed_dropout): Dropout(p=0.0, inplace=False)
    (layers): ModuleList(
      (0-23): 24 x PhiDecoderLayer(
        (self_attn): PhiAttention(
          (q_proj): lora.Linear(
            (base_layer): Linear(in_features=2048, out_features=2048, bias=True)
            (lora_dropout): ModuleDict(
              (default): Dropout(p=0.05, inplace=False)
            )
            (lora_A): ModuleDict(
              (default): Linear(in_features=2048, out_features=32, bias=False)
            )
            (lora_B): ModuleDict(
              (default): Linear(in_features=32, out_features=2048, bias=False)
            )
            (lora_embedding_A): ParameterDict()
            (lora_embedding_B): ParameterDict()
            (lora_magnitude_vector): ModuleDict()
          )
          (k_proj): Linear(in_features=2048, out_features=2048, bias=True)
          (v_proj): lora.Linear(
            (base_layer): Li

In [54]:
loading_class = AutoModelForCausalLMWithValueHead if 'causal' in sft_model.config.architectures[0].lower() else AutoModelForSeq2SeqLMWithValueHead
ppo_model = loading_class.from_pretrained(
    rl_peft_model, torch_dtype=PRECISION, is_trainable=True, device_map='auto'
)

AutoModelForCausalLMWithValueHead(
  (pretrained_model): PeftModelForCausalLM(
    (base_model): LoraModel(
      (model): PhiForCausalLM(
        (model): PhiModel(
          (embed_tokens): Embedding(51200, 2048)
          (embed_dropout): Dropout(p=0.0, inplace=False)
          (layers): ModuleList(
            (0-23): 24 x PhiDecoderLayer(
              (self_attn): PhiAttention(
                (q_proj): lora.Linear(
                  (base_layer): Linear(in_features=2048, out_features=2048, bias=True)
                  (lora_dropout): ModuleDict(
                    (default): Dropout(p=0.05, inplace=False)
                  )
                  (lora_A): ModuleDict(
                    (default): Linear(in_features=2048, out_features=32, bias=False)
                  )
                  (lora_B): ModuleDict(
                    (default): Linear(in_features=32, out_features=2048, bias=False)
                  )
                  (lora_embedding_A): ParameterDict()
               

In [ ]:
N_EVAL_SAMPLES = int(len(dataset['test']) * 0.2)
print(N_EVAL_SAMPLES)
start = time.time()
peft_checkpoint_generation = quantitative_comparison(
    ppo_model,
    dataset,
    tokenizer,
    n_samples_to_evaluate=N_EVAL_SAMPLES,
    batch_size=20,
    device=DEVICE,
)
baseline_model_generation = quantitative_comparison(
    sft_model,
    dataset,
    tokenizer,
    n_samples_to_evaluate=N_EVAL_SAMPLES,
    batch_size=20,
    device=DEVICE,
)

end = time.time()

duration = end - start
print(
    f"Evaluating N={N_EVAL_SAMPLES} samples took {round(duration, 2)} seconds to execute."
)

human_baseline_answer = dataset["test"][0:N_EVAL_SAMPLES]["summary"]

zipped_summaries = list(
    zip(human_baseline_answer, peft_checkpoint_generation, baseline_model_generation)
)

In [57]:
# peft_checkpoint_generation

['\n    Summarize the following reddit post.\n\n    Hello hello everybody. I hope this isn\'t too trivial of a question to ask on here, but I\'ve been feeling a bit out of my depth when it comes to this situation (I\'ve had only one relationship before, and for many reasons, it was out of the ordinary).\n\nOkay! So, a couple of weeks ago, I started talking to this guy on Facebook, through a student group that we were both part of. I thought he was sort of cute, so I sent him a PM just to talk, etc, etc. We\'re both transfer students at the same school, so I knew that we could eventually meet in person once we both moved on-campus. So, we did, and we hung out maybe twice, just as friends.\n\nOkay. So, everything is going pretty well. We talk over Facebook and Snapchat, whatever. So, Saturday night, I was just hanging out with people and kind of being bored, when I got a Snapchat from him asking what I was doing. I asked if he wanted to hang out, so we did. \n\nWe ended up smoking pot (t

In [46]:
df = pd.DataFrame(
    zipped_summaries,
    columns=[
        "human_baseline_answer",
        "peft_checkpoint_generation",
        "baseline_model_generation",
    ],
)
df.head()
print(df.shape)

(131, 3)


In [53]:
rouge = evaluate.load("rouge")

original_model_results = rouge.compute(
    predictions=baseline_model_generation,
    references=human_baseline_answer[0 : len(baseline_model_generation)],
    use_aggregator=True,
    use_stemmer=True,
)

peft_model_results = rouge.compute(
    predictions=peft_checkpoint_generation,
    references=human_baseline_answer[0 : len(peft_checkpoint_generation)],
    use_aggregator=True,
    use_stemmer=True,
)

original_model_results = round_dictionary_values(original_model_results)
# instruct_model_results = round_dictionary_values(instruct_model_results)
peft_model_results = round_dictionary_values(peft_model_results)
print("SFT MODEL:")
print(original_model_results)
# print('INSTRUCT MODEL:')
# print(instruct_model_results)
print("PEFT MODEL:")
print(peft_model_results)

SFT MODEL:
{'rouge1': 0.16, 'rouge2': 0.06, 'rougeL': 0.11, 'rougeLsum': 0.12}
PEFT MODEL:
{'rouge1': 0.16, 'rouge2': 0.06, 'rougeL': 0.11, 'rougeLsum': 0.12}


In [ ]:
COMMON_OUTPUT_PATHS = COMMON_OUTPUT_PATHS.format(RUN_ID=RM_TRAIN_DATA_RUN_ID if RM_TRAIN_DATA_RUN_ID is not None else RUN_ID)
METRICS_PATH = METRICS_PATH.format(COMMON_OUTPUT_PATHS=COMMON_OUTPUT_PATHS)

if not os.path.exists(METRICS_PATH):
    os.makedirs(METRICS_PATH)

data_path = f'{METRICS_PATH}/rl-results.json'

results_dict = {'sft-model': original_model_results, 'rl-model': peft_model_results}

with open(data_path, 'w') as file:
    json.dump(results_dict, file)
    
print("Absolute percentage improvement of PPO MODEL over SFT MODEL.")

improvement = np.array(list(peft_model_results.values())) - np.array(
    list(original_model_results.values())
)
for key, value in zip(peft_model_results.keys(), improvement):
    print(f'{key}: {value*100:.2f}%')

In [ ]:
# add one more eval, for higher rated summaries.

## END